# 1. MODEL TRAINING

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pandas as pd
from pathlib import Path
import pickle
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,Conv2D,MaxPooling2D

In [2]:
Data_dir = '../input/chest-xray/chest_xray'
Categories = ["Affected" , "Normal"]
for i in Categories:
    path = os.path.join(Data_dir,i)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)

In [3]:
Img_size = 224
new_array = cv2.resize(img_array, (Img_size,Img_size))

In [4]:
training_data = []
def create_training_data():
    for i in Categories:
        path = os.path.join(Data_dir,i)
        class_num = Categories.index(i) ## 0 for affected 1 for Normal
        for img in os.listdir(path):
            img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
            new_array = cv2.resize(img_array, (Img_size,Img_size))
            training_data.append([new_array, class_num])
create_training_data()

In [5]:
import random
random.shuffle(training_data)

In [6]:
x = []
y = []

for features,label in training_data:
    x.append(features)
    y.append(label)
x =np.array(x).reshape(-1,Img_size,Img_size, 1)

In [7]:
pickle_out = open("x.pickle","wb")
pickle.dump(x, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [8]:
pickle_in = open("x.pickle", "rb")
x = pickle.load(pickle_in)

In [9]:
x = pickle.load(open("x.pickle","rb"))
y = pickle.load(open("y.pickle","rb"))

x = x/255.0

model = Sequential()
model.add(Conv2D(64,(3,3),input_shape=x.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss="binary_crossentropy",optimizer="adam",metrics=['accuracy'])
x= np.array(x)
y= np.array(y)
model.fit(x, y, batch_size = 32,epochs=10,validation_split=0.3)


2022-11-21 15:21:35.728741: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-21 15:21:35.729628: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-21 15:21:36.067412: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-21 15:21:36.068198: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-21 15:21:36.068998: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

Epoch 1/10


2022-11-21 15:21:46.547294: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


129/129 [==============================] - 24s 84ms/step - loss: 0.2976 - accuracy: 0.8705 - val_loss: 0.1723 - val_accuracy: 0.9340
Epoch 2/10
129/129 [==============================] - 9s 69ms/step - loss: 0.1560 - accuracy: 0.9432 - val_loss: 0.1367 - val_accuracy: 0.9442
Epoch 3/10
129/129 [==============================] - 9s 69ms/step - loss: 0.1166 - accuracy: 0.9568 - val_loss: 0.1365 - val_accuracy: 0.9448
Epoch 4/10
129/129 [==============================] - 9s 70ms/step - loss: 0.1075 - accuracy: 0.9624 - val_loss: 0.1294 - val_accuracy: 0.9505
Epoch 5/10
129/129 [==============================] - 9s 70ms/step - loss: 0.0807 - accuracy: 0.9697 - val_loss: 0.1572 - val_accuracy: 0.9385
Epoch 6/10
129/129 [==============================] - 9s 69ms/step - loss: 0.0607 - accuracy: 0.9773 - val_loss: 0.1649 - val_accuracy: 0.9363
Epoch 7/10
129/129 [==============================] - 9s 69ms/step - loss: 0.0325 - accuracy: 0.9893 - val_loss: 0.1522 - val_accuracy: 0.9533
Epoch 8/1

In [10]:
model.save('Pneumonia_detection.h5')